In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
#Загрузка данных и создание объединенного Dataset
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test['SalePrice'] = np.nan
df = pd.concat([df_train, df_test], ignore_index=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       2919 non-null   object 
 1   LotFrontage    2919 non-null   float64
 2   LotArea        2919 non-null   int64  
 3   Street         2919 non-null   object 
 4   LotShape       2919 non-null   object 
 5   LandContour    2919 non-null   object 
 6   Utilities      2919 non-null   object 
 7   LotConfig      2919 non-null   object 
 8   LandSlope      2919 non-null   object 
 9   Neighborhood   2919 non-null   object 
 10  Condition1     2919 non-null   object 
 11  Condition2     2919 non-null   object 
 12  BldgType       2919 non-null   object 
 13  HouseStyle     2919 non-null   object 
 14  OverallQual    2919 non-null   int64  
 15  OverallCond    2919 non-null   int64  
 16  YearBuilt      2919 non-null   int64  
 17  YearRemodAdd   2919 non-null   int64  
 18  RoofStyl

In [ ]:
#Работа с пропусками

In [3]:
#По похожим объектам по decisionCol столбцу определяет самые частые значения nullCol столбца
#!!! decisionCol столбец БЕЗ NULL значений
def getValueBasedOnTheSecondColumn(data, nullCol, decisionCol):
    fill_null_dict = data.groupby(by=decisionCol)[nullCol].apply(lambda obj: Counter(obj).most_common(1)[0][0])
    data[nullCol] = data[nullCol].fillna(data[data[nullCol].isnull()][decisionCol].map(fill_null_dict))
    return data

In [4]:
#Если значение в decisionCol != decisionValue, то nullCol = [макс. по частоте в nullCol], иначе nullValue
def getValueBasedOnTheThreshold(data, decisionCol, decisionValue, nullCol, nullValue):
    most_freq_val = Counter(df[nullCol]).most_common(1)[0][0]
    data.loc[df[(df[decisionCol]!=decisionValue)&(df[nullCol].isnull())].index, nullCol] = most_freq_val
    data[nullCol] = data[nullCol].fillna(nullValue)
    return data

In [5]:
#Вывод несбалансированных колонок
def getUnbalancedColumns(data, threshold):
    del_columns = []
    for col_name in data.columns:
        freq = data[col_name].value_counts().max() * 100 / data.shape[0]
        if freq >= threshold:
            del_columns.append(col_name)
    return del_columns

In [6]:
#Замена данных
df = df.drop(['Alley','PoolQC','MiscFeature','Fence','Id','MSSubClass'], axis=1)
df = getValueBasedOnTheSecondColumn(df, 'MSZoning', 'Neighborhood')
df['LotFrontage'] = df['LotFrontage'].fillna(0)
df = getValueBasedOnTheSecondColumn(df, 'Utilities', 'MSZoning')
df['Exterior1st'] = df['Exterior1st'].fillna('Other')
df['Exterior2nd'] = df['Exterior2nd'].fillna('Other')
#Площадь кладки
df = getValueBasedOnTheThreshold(df, 'MasVnrArea', 0, 'MasVnrType', 'None')
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

df = getValueBasedOnTheThreshold(df,'TotalBsmtSF',0,'BsmtQual','NA')
df = getValueBasedOnTheThreshold(df,'TotalBsmtSF',0,'BsmtCond','NA')
df = getValueBasedOnTheThreshold(df,'TotalBsmtSF',0,'BsmtExposure','NA')
df = getValueBasedOnTheThreshold(df,'TotalBsmtSF',0,'BsmtFinType1','NA')
df['BsmtFinSF1'] = df['BsmtFinSF1'].fillna(0)
df = getValueBasedOnTheThreshold(df,'TotalBsmtSF',0,'BsmtFinType2','NA')
df['BsmtFinSF2'] = df['BsmtFinSF2'].fillna(0)
df['BsmtUnfSF'] = df['BsmtUnfSF'].fillna(0)
df['TotalBsmtSF'] = df['TotalBsmtSF'].fillna(0)
df['Electrical'] = df['Electrical'].fillna('SBrkr')
df['BsmtHalfBath'] = df['BsmtHalfBath'].fillna(0)
df['BsmtFullBath'] = df['BsmtFullBath'].fillna(0)
#Гараж
df.loc[2576, 'GarageType'] = np.nan
df.loc[2126, ['GarageType','GarageCars','GarageArea']] = np.full(3, np.nan)
df['GarageType'] = df['GarageType'].fillna('NA')
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df[df['GarageYrBlt'].isnull()]['YearBuilt'])
df['GarageFinish'] = df['GarageFinish'].fillna('NA')
df['GarageCars'] = df['GarageCars'].fillna(0)
df['GarageArea'] = df['GarageArea'].fillna(0)
df['GarageQual'] = df['GarageQual'].fillna('NA')
df['GarageCond'] = df['GarageCond'].fillna('NA')

df['KitchenQual'] = df['KitchenQual'].fillna('TA')
df['Functional'] = df['Functional'].fillna('Typ')
df['FireplaceQu'] = df['FireplaceQu'].fillna('NA')
df = getValueBasedOnTheSecondColumn(df,'SaleType','SaleCondition')

In [280]:
#Работа с признаками

In [7]:
#Замена категориальных признаков на непр.
qual_change_map = {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5, 'No':1, 'Mn':2, 
                     'Av':3, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}
df['ExterQual'] = df['ExterQual'].map(qual_change_map)
df['ExterCond'] = df['ExterCond'].map(qual_change_map)
df['BsmtQual'] = df['BsmtQual'].map(qual_change_map)
df['BsmtCond'] = df['BsmtCond'].map(qual_change_map)
df['BsmtExposure'] = df['BsmtExposure'].map(qual_change_map)
df['BsmtFinType1'] = df['BsmtFinType1'].map(qual_change_map)
df['BsmtFinType2'] = df['BsmtFinType2'].map(qual_change_map)
df['HeatingQC'] = df['HeatingQC'].map(qual_change_map)
df['CentralAir'] = df['CentralAir'].map({'Y':1, 'N':0})
df['KitchenQual'] = df['KitchenQual'].map(qual_change_map)
df['FireplaceQu'] = df['FireplaceQu'].map(qual_change_map)
df['GarageQual'] = df['GarageQual'].map(qual_change_map)
df['GarageCond'] = df['GarageCond'].map(qual_change_map)

In [8]:
#Создание новых признаков
df['HouseAge'] = 2025 - df['YearBuilt']
df['HouseRemodAge'] = 2025 - df['YearRemodAdd']
df['GarageAge'] = 2025 - df['GarageYrBlt']
df['FullHouseSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['TotalBsmtSF']

In [ ]:
#Тест моделей

In [378]:
def runModel(threshold_begin, threshold_end, model, dummy_df):
    for threshold in range(threshold_begin, threshold_end+1):
        del_cols = getUnbalancedColumns(dummy_df, threshold)
        df_droped = dummy_df.drop(del_cols, axis=1)
        X = df_droped.drop('SalePrice', axis=1)
        y = df_droped['SalePrice']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
        model.fit(X_train, y_train)
        y_predict = model.predict(X_test)
        print('Порог =', threshold, 'Ошибка модели =', mean_absolute_error(y_test, y_predict))
        print(model.best_estimator_['model'])

In [9]:
#Обратное разделение на Train и Test
dummy_df = pd.get_dummies(df)
dummy_train = dummy_df[dummy_df['SalePrice'].notnull()]
dummy_test = dummy_df[dummy_df['SalePrice'].isnull()]

In [390]:
#Градиентный бустинг
model = GradientBoostingRegressor()
pipe_model = Pipeline([('model', model)])
grid_params = {'model__n_estimators': [108,109,110,111,112],
               'model__max_depth': [3,4,5]}
grid_model = GridSearchCV(pipe_model, 
                          param_grid=grid_params, 
                          scoring='neg_mean_absolute_error', 
                          cv=5, 
                          verbose=0)

In [391]:
runModel(98, 98, grid_model, dummy_train)

Порог = 98 Ошибка модели = 14841.763055967383
GradientBoostingRegressor(max_depth=4, n_estimators=109)


In [ ]:
#Получение результата

In [11]:
del_cols = getUnbalancedColumns(dummy_train, 98)
dummy_train = dummy_train.drop(del_cols, axis=1)
dummy_test = dummy_test.drop(del_cols, axis=1)

In [14]:
dummy_test = dummy_test.drop('SalePrice', axis=1)

In [15]:
X = dummy_train.drop('SalePrice', axis=1)
y = dummy_train['SalePrice']

best_model = GradientBoostingRegressor(n_estimators=109, max_depth=4)
best_model.fit(X, y)

GradientBoostingRegressor(max_depth=4, n_estimators=109)

In [16]:
y_predict = best_model.predict(dummy_test)

In [28]:
res_data = pd.concat([pd.Series(df_test['Id']), pd.Series(y_predict, name='SalePrice')], axis=1)

In [31]:
res_data.to_csv('res.csv', index=False)